# IV Rank-Based Short Straddle Strategy

This notebook demonstrates an options strategy that:
- **Sells 30-60 day ATM straddles** when IV conditions are favorable
- **Entry Rules**: Enter when 90-day IV Rank > 60% OR < 30%
- **Exit Rules**: Profit target, stop loss, or DTE-based exit

Additionally, this notebook showcases the **new Phase C Analytics**:
- **Monte Carlo Simulation** for confidence intervals and risk estimation
- **Scenario Testing** for stress testing and sensitivity analysis

## 1. Setup and Imports

In [ ]:
import sys
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Add backtester to path
sys.path.insert(0, str(Path('..') / 'code'))

from datetime import datetime, timedelta
import pandas as pd
import numpy as np

# Core backtester imports
from backtester.data.dolt_adapter import DoltAdapter
from backtester.engine.backtest_engine import BacktestEngine
from backtester.engine.execution import ExecutionModel
from backtester.engine.data_stream import DataStream
from backtester.strategies.strategy import Strategy
from backtester.structures.straddle import ShortStraddle

# Analytics imports
from backtester.analytics.metrics import PerformanceMetrics
from backtester.analytics.risk import RiskAnalytics
from backtester.analytics.visualization import Visualization

# NEW Phase C: Monte Carlo and Scenario Testing
from backtester.analytics import (
    MonteCarloSimulator, SimulationResult, ConfidenceInterval,
    ScenarioTester, Scenario, ScenarioResult, SensitivityResult,
    STRESS_SCENARIOS, HISTORICAL_SCENARIOS,
    get_predefined_scenario, list_available_scenarios
)

# Visualization
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

print('All imports successful!')
print(f'Notebook created: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}')

## 2. Configuration

In [ ]:
# Database and backtest configuration
DB_PATH = '/Users/janussuk/Desktop/AI Projects/OptionsBacktester2/dolt_data/options'
UNDERLYING = 'SPY'
START_DATE = datetime(2023, 1, 4)
END_DATE = datetime(2025, 10, 29)
INITIAL_CAPITAL = 100000.0

# Strategy parameters
MIN_DTE = 30  # Minimum days to expiration
MAX_DTE = 60  # Maximum days to expiration
IV_LOOKBACK = 90  # Days for IV rank calculation
IV_RANK_HIGH = 60  # Enter when IV rank > 60%
IV_RANK_LOW = 30   # Or enter when IV rank < 30%

print(f'Configuration:')
print(f'  Underlying: {UNDERLYING}')
print(f'  Date Range: {START_DATE.date()} to {END_DATE.date()}')
print(f'  Initial Capital: ${INITIAL_CAPITAL:,.0f}')
print(f'  DTE Range: {MIN_DTE}-{MAX_DTE} days')
print(f'  IV Rank Lookback: {IV_LOOKBACK} days')
print(f'  Entry Condition: IV Rank > {IV_RANK_HIGH}% OR < {IV_RANK_LOW}%')

## 3. Database Connection

In [ ]:
# Connect to database
print(f'Connecting to Dolt database at: {DB_PATH}')
adapter = DoltAdapter(DB_PATH)
adapter.connect()

# Verify database
tables = adapter.get_tables()
print(f'Database connected!')
print(f'Available tables: {tables}')

# Check data availability
date_range = adapter.get_date_range(UNDERLYING)
print(f'Data range for {UNDERLYING}: {date_range[0]} to {date_range[1]}')

## 4. Strategy Implementation

### IV Rank-Based Short Straddle Strategy

**Entry Rules:**
- Sell ATM straddle when 90-day IV Rank > 60% (high IV - expecting mean reversion)
- OR sell ATM straddle when 90-day IV Rank < 30% (low IV - range-bound market)
- DTE between 30-60 days

**Exit Rules:**
- 50% profit target
- 200% stop loss (2x premium received)
- Exit at 7 DTE or earlier

In [ ]:
class IVRankStraddleStrategy(Strategy):
    """
    IV Rank-Based Short Straddle Strategy.
    
    Sells 30-60 day ATM straddles when:
    - 90-day IV Rank > 60% (high IV regime - expect mean reversion)
    - OR 90-day IV Rank < 30% (low IV regime - range-bound market)
    """
    
    def __init__(self, name='IVRankStraddle', initial_capital=100000.0,
                 min_dte=30, max_dte=60, iv_lookback=90,
                 iv_rank_high=60, iv_rank_low=30):
        super().__init__(
            name=name,
            initial_capital=initial_capital,
            position_limits={
                'max_positions': 5,
                'max_total_delta': 5000.0,
                'max_total_vega': 3000.0,
                'max_capital_utilization': 0.8,
                'max_single_position_size': initial_capital * 0.3
            }
        )
        
        # Strategy parameters
        self.min_dte = min_dte
        self.max_dte = max_dte
        self.iv_lookback = iv_lookback
        self.iv_rank_high = iv_rank_high
        self.iv_rank_low = iv_rank_low
        
        # Exit parameters
        self.exit_dte = 7
        self.profit_target_pct = 0.50  # 50% of max profit
        self.stop_loss_pct = 2.00      # 200% of max profit (loss)
        self.contracts_per_trade = 1
        
        # Track IV history for rank calculation
        self.iv_history = []
        self.entry_reasons = []  # Track why we entered
        
    def calculate_iv_rank(self, current_iv):
        """Calculate IV rank (percentile over lookback period)."""
        if len(self.iv_history) < 20:
            return 50.0  # Default to mid-range if insufficient data
        
        iv_min = min(self.iv_history)
        iv_max = max(self.iv_history)
        
        if iv_max == iv_min:
            return 50.0
        
        # IV Rank formula: (Current IV - Min IV) / (Max IV - Min IV) * 100
        iv_rank = ((current_iv - iv_min) / (iv_max - iv_min)) * 100
        return iv_rank
    
    def should_enter(self, market_data):
        """Determine if we should enter a new position."""
        # Update IV history
        current_iv = market_data.get('iv', 0.20)
        self.iv_history.append(current_iv)
        if len(self.iv_history) > self.iv_lookback:
            self.iv_history.pop(0)
        
        # Calculate IV rank
        iv_rank = self.calculate_iv_rank(current_iv)
        
        # Entry conditions: IV Rank > 60% OR < 30%
        high_iv_entry = iv_rank > self.iv_rank_high
        low_iv_entry = iv_rank < self.iv_rank_low
        
        # Position limit check
        max_positions_ok = self.num_open_positions < self._position_limits['max_positions']
        
        should_enter = (high_iv_entry or low_iv_entry) and max_positions_ok
        
        if should_enter:
            reason = f"High IV ({iv_rank:.1f}%)" if high_iv_entry else f"Low IV ({iv_rank:.1f}%)"
            self.entry_reasons.append(reason)
        
        return should_enter
    
    def should_exit(self, structure, market_data):
        """Determine if we should exit a position."""
        # Calculate days to expiration
        current_date = market_data.get('date', datetime.now())
        if hasattr(structure, 'call_option'):
            expiration = structure.call_option.expiration
            dte = (expiration - current_date).days
        else:
            dte = 999
        
        # Calculate P&L percentage
        try:
            current_pnl = structure.calculate_pnl()
            max_profit = structure.max_profit if hasattr(structure, 'max_profit') else abs(structure.net_premium)
            
            if max_profit > 0:
                pnl_pct = current_pnl / max_profit
            else:
                pnl_pct = 0.0
        except:
            pnl_pct = 0.0
        
        # Exit conditions
        profit_target_hit = pnl_pct >= self.profit_target_pct
        stop_loss_hit = pnl_pct <= -self.stop_loss_pct
        dte_exit = dte <= self.exit_dte
        
        return profit_target_hit or stop_loss_hit or dte_exit
    
    def create_structure(self, market_data):
        """Create a short straddle structure from market data."""
        option_chain = market_data.get('option_chain')
        if option_chain is None or option_chain.empty:
            return None
        
        current_date = market_data.get('date', datetime.now())
        spot = market_data.get('spot', 0.0)
        
        # Calculate DTE for each option
        option_chain = option_chain.copy()
        option_chain['dte'] = (pd.to_datetime(option_chain['expiration']) - current_date).dt.days
        
        # Filter by target DTE range (30-60 days)
        target_dte_options = option_chain[
            (option_chain['dte'] >= self.min_dte) & 
            (option_chain['dte'] <= self.max_dte)
        ]
        
        if target_dte_options.empty:
            return None
        
        # Find ATM strike (closest to spot)
        target_dte_options['strike_diff'] = abs(target_dte_options['strike'] - spot)
        atm_strike = target_dte_options.loc[target_dte_options['strike_diff'].idxmin(), 'strike']
        
        # Get target expiration (closest to 45 DTE - middle of 30-60 range)
        target_dte = 45
        target_dte_options['dte_diff'] = abs(target_dte_options['dte'] - target_dte)
        target_expiration = target_dte_options.loc[target_dte_options['dte_diff'].idxmin(), 'expiration']
        
        # Filter for ATM strike and target expiration
        atm_options = option_chain[
            (option_chain['strike'] == atm_strike) &
            (option_chain['expiration'] == target_expiration)
        ]
        
        # Get call and put
        calls = atm_options[atm_options['call_put'] == 'Call']
        puts = atm_options[atm_options['call_put'] == 'Put']
        
        if calls.empty or puts.empty:
            return None
        
        call_data = calls.iloc[0]
        put_data = puts.iloc[0]
        
        # Use mid price
        call_price = (call_data['bid'] + call_data['ask']) / 2.0 if call_data['ask'] > 0 else call_data['bid']
        put_price = (put_data['bid'] + put_data['ask']) / 2.0 if put_data['ask'] > 0 else put_data['bid']
        
        # Create short straddle
        try:
            straddle = ShortStraddle.create(
                underlying=UNDERLYING,
                strike=atm_strike,
                expiration=pd.to_datetime(target_expiration).to_pydatetime(),
                call_price=call_price,
                put_price=put_price,
                quantity=self.contracts_per_trade,
                entry_date=current_date,
                underlying_price=spot,
                call_iv=call_data.get('vol', 0.20),
                put_iv=put_data.get('vol', 0.20)
            )
            return straddle
        except Exception as e:
            return None


# Create strategy instance
strategy = IVRankStraddleStrategy(
    name='IVRank_30_60_Straddle',
    initial_capital=INITIAL_CAPITAL,
    min_dte=MIN_DTE,
    max_dte=MAX_DTE,
    iv_lookback=IV_LOOKBACK,
    iv_rank_high=IV_RANK_HIGH,
    iv_rank_low=IV_RANK_LOW
)

print(f'Strategy created: {strategy.name}')
print(f'\nParameters:')
print(f'  - DTE Range: {strategy.min_dte}-{strategy.max_dte} days')
print(f'  - IV Lookback: {strategy.iv_lookback} days')
print(f'  - Entry: IV Rank > {strategy.iv_rank_high}% OR < {strategy.iv_rank_low}%')
print(f'  - Exit DTE: {strategy.exit_dte} days')
print(f'  - Profit Target: {strategy.profit_target_pct:.0%}')
print(f'  - Stop Loss: {strategy.stop_loss_pct:.0%}')
print(f'  - Max Positions: {strategy._position_limits["max_positions"]}')

## 5. Run Backtest

In [ ]:
# Create data stream
print(f'Creating data stream for {UNDERLYING} from {START_DATE.date()} to {END_DATE.date()}')
data_stream = DataStream(
    data_source=adapter,
    start_date=START_DATE,
    end_date=END_DATE,
    underlying=UNDERLYING,
    dte_range=(MIN_DTE, MAX_DTE),
    cache_enabled=True,
    skip_missing_data=True
)

print(f'Data stream created: {data_stream.num_trading_days} trading days')

# Create execution model
execution = ExecutionModel(
    commission_per_contract=0.65,
    slippage_pct=0.001
)

# Create backtest engine
engine = BacktestEngine(
    strategy=strategy,
    data_stream=data_stream,
    execution_model=execution,
    initial_capital=INITIAL_CAPITAL
)

print(f'Backtest engine ready')
print(f'  - Initial Capital: ${INITIAL_CAPITAL:,.2f}')
print(f'  - Commission: ${execution.commission_per_contract:.2f} per contract')

In [ ]:
# Run the backtest
print('=' * 80)
print('RUNNING BACKTEST')
print('=' * 80)

results = engine.run()

print('=' * 80)
print('BACKTEST COMPLETE')
print('=' * 80)
print(f'Final Equity: ${results["final_equity"]:,.2f}')
print(f'Total Return: {results["total_return"]:.2%}')
print(f'Total Trades: {results["num_trades"]}')
print(f'Entries: {results["num_entries"]} | Exits: {results["num_exits"]}')

## 6. Performance Metrics

In [ ]:
# Calculate all metrics
metrics = engine.calculate_metrics(risk_free_rate=0.04)

# Display performance metrics
print('=' * 80)
print('PERFORMANCE METRICS')
print('=' * 80)
perf = metrics['performance']
print(f"Total Return:           {perf.get('total_return_pct', 0):.2f}%")
print(f"Annualized Return:      {perf.get('annualized_return', 0):.2%}")
print(f"Volatility (Annual):    {perf.get('volatility', 0):.2%}")
print(f"Sharpe Ratio:           {perf.get('sharpe_ratio', 0):.3f}")
print(f"Sortino Ratio:          {perf.get('sortino_ratio', 0):.3f}")
print(f"Calmar Ratio:           {perf.get('calmar_ratio', 0):.3f}")
print(f"Max Drawdown:           {perf.get('max_drawdown', 0):.2%}")

print('\n' + '=' * 80)
print('TRADE METRICS')
print('=' * 80)
print(f"Total Trades:           {perf.get('total_trades', 0)}")
print(f"Win Rate:               {perf.get('win_rate', 0):.2f}%")
print(f"Profit Factor:          {perf.get('profit_factor', 0):.3f}")
print(f"Expectancy:             ${perf.get('expectancy', 0):,.2f}")
print(f"Average Win:            ${perf.get('average_win', 0):,.2f}")
print(f"Average Loss:           ${perf.get('average_loss', 0):,.2f}")

print('\n' + '=' * 80)
print('RISK METRICS')
print('=' * 80)
risk = metrics['risk']
print(f"VaR (95%, Historical):  {risk.get('var_95_historical', 0):.4f}")
print(f"CVaR (95%):             {risk.get('cvar_95', 0):.4f}")
print(f"Skewness:               {risk.get('skewness', 0):.4f}")
print(f"Kurtosis:               {risk.get('kurtosis', 0):.4f}")

# Store for later use
equity_curve = results['equity_curve']
trade_log = results['trade_log']

## 7. Monte Carlo Simulation (Phase C)

Use Monte Carlo simulation to:
- Generate confidence intervals for performance metrics
- Estimate VaR and CVaR through simulation
- Analyze the distribution of potential outcomes

In [ ]:
# Calculate daily returns from equity curve
equity_series = pd.Series(equity_curve['equity'].values, index=pd.to_datetime(equity_curve['date']))
daily_returns = equity_series.pct_change().dropna()

print(f'Daily returns calculated: {len(daily_returns)} observations')
print(f'Mean daily return: {daily_returns.mean():.4%}')
print(f'Daily volatility: {daily_returns.std():.4%}')

In [ ]:
# Run Monte Carlo simulation using Bootstrap method
print('Running Monte Carlo Simulation (Bootstrap)...')
print('=' * 80)

mc_result = MonteCarloSimulator.simulate_returns_bootstrap(
    returns=daily_returns.values,
    num_simulations=10000,
    num_periods=252,  # 1 year forward
    random_seed=42
)

print(f'Simulation complete!')
print(f'  Simulations: {mc_result.num_simulations:,}')
print(f'  Periods per path: {mc_result.num_periods}')
print(f'  Method: {mc_result.method}')

In [ ]:
# Calculate confidence intervals for key metrics
print('\nConfidence Intervals (95%):')
print('=' * 80)

confidence_intervals = MonteCarloSimulator.calculate_all_confidence_intervals(
    mc_result, 
    confidence_level=0.95
)

for metric, ci in confidence_intervals.items():
    print(f'{metric:20s}: [{ci.lower_bound:8.2%}, {ci.upper_bound:8.2%}]  (mean: {ci.mean:8.2%})')

In [ ]:
# Estimate VaR and CVaR via Monte Carlo
print('\nMonte Carlo VaR/CVaR Estimation:')
print('=' * 80)

var_metrics = MonteCarloSimulator.estimate_var_at_multiple_levels(
    mc_result,
    confidence_levels=[0.90, 0.95, 0.99]
)

for level, var_data in var_metrics.items():
    print(f"\n{level}:")
    print(f"  VaR:  {var_data['var']:.2%}")
    print(f"  CVaR: {var_data['cvar']:.2%}")

In [ ]:
# Analyze return distribution
print('\nReturn Distribution Analysis:')
print('=' * 80)

dist_analysis = MonteCarloSimulator.analyze_return_distribution(mc_result)

print(f"Mean Return:     {dist_analysis['mean']:.2%}")
print(f"Median Return:   {dist_analysis['median']:.2%}")
print(f"Std Dev:         {dist_analysis['std']:.2%}")
print(f"Skewness:        {dist_analysis['skewness']:.4f}")
print(f"Kurtosis:        {dist_analysis['kurtosis']:.4f}")
print(f"Min Return:      {dist_analysis['min']:.2%}")
print(f"Max Return:      {dist_analysis['max']:.2%}")
print(f"\nPercentiles:")
for pct, val in dist_analysis['percentiles'].items():
    print(f"  {pct}: {val:.2%}")

In [ ]:
# Probability of reaching targets
print('\nProbability Analysis:')
print('=' * 80)

targets = [0.10, 0.20, 0.30, 0.50]  # 10%, 20%, 30%, 50% returns
for target in targets:
    prob = MonteCarloSimulator.calculate_probability_of_target(mc_result, target)
    print(f"P(Return >= {target:5.0%}): {prob:.1%}")

# Probability of loss
prob_loss = MonteCarloSimulator.calculate_probability_of_target(mc_result, 0.0)
print(f"\nP(Positive Return): {prob_loss:.1%}")
print(f"P(Loss):            {100 - prob_loss:.1%}")

In [ ]:
# Visualize Monte Carlo simulation results
# Convert to DataFrame for easier plotting
mc_df = MonteCarloSimulator.paths_to_dataframe(mc_result, sample_paths=100)

fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        'Sample Simulation Paths (100 of 10,000)',
        'Final Return Distribution',
        'Cumulative Equity Paths',
        'Drawdown Distribution'
    )
)

# Sample paths
for col in mc_df.columns[:100]:
    cumulative = (1 + mc_df[col]).cumprod()
    fig.add_trace(
        go.Scatter(y=cumulative.values, mode='lines', 
                   line=dict(width=0.5, color='rgba(100,100,200,0.3)'),
                   showlegend=False),
        row=1, col=1
    )

# Final return distribution
final_returns = mc_result.total_returns
fig.add_trace(
    go.Histogram(x=final_returns, nbinsx=50, name='Final Returns',
                 marker_color='steelblue'),
    row=1, col=2
)

# Add VaR lines
var_95 = np.percentile(final_returns, 5)
fig.add_vline(x=var_95, line_dash='dash', line_color='red', 
              annotation_text=f'VaR 95%: {var_95:.1%}', row=1, col=2)

# Cumulative equity paths (median and percentiles)
cumulative_paths = (1 + mc_df).cumprod()
median_path = cumulative_paths.median(axis=1)
p5 = cumulative_paths.quantile(0.05, axis=1)
p95 = cumulative_paths.quantile(0.95, axis=1)

fig.add_trace(
    go.Scatter(y=median_path.values, mode='lines', name='Median',
               line=dict(color='blue', width=2)),
    row=2, col=1
)
fig.add_trace(
    go.Scatter(y=p95.values, mode='lines', name='95th Percentile',
               line=dict(color='green', dash='dash')),
    row=2, col=1
)
fig.add_trace(
    go.Scatter(y=p5.values, mode='lines', name='5th Percentile',
               line=dict(color='red', dash='dash')),
    row=2, col=1
)

# Drawdown distribution
dd_analysis = MonteCarloSimulator.analyze_drawdown_distribution(mc_result)
fig.add_trace(
    go.Histogram(x=dd_analysis['max_drawdowns'], nbinsx=50, name='Max Drawdowns',
                 marker_color='indianred'),
    row=2, col=2
)

fig.update_layout(
    height=700,
    title_text='Monte Carlo Simulation Analysis',
    showlegend=True
)

fig.show()

## 8. Scenario Testing / Stress Testing (Phase C)

Use scenario testing to:
- Stress test the portfolio against historical market events
- Analyze sensitivity to market factors
- Perform what-if analysis

In [ ]:
# List available scenarios
print('Available Predefined Scenarios:')
print('=' * 80)

scenarios = list_available_scenarios()
print('\nStress Scenarios:')
for name in scenarios['stress']:
    scenario = STRESS_SCENARIOS[name]
    print(f"  - {name}: spot={scenario.spot_change:+.0%}, vol={scenario.vol_change:+.0%}")

print('\nHistorical Scenarios:')
for name in scenarios['historical']:
    scenario = HISTORICAL_SCENARIOS[name]
    print(f"  - {name}: spot={scenario.spot_change:+.0%}, vol={scenario.vol_change:+.0%}")

In [ ]:
# Define portfolio Greeks (typical short straddle position)
# These would normally come from your actual position
portfolio_greeks = {
    'delta': -15.0,    # Slightly short delta (typical for short straddle near ATM)
    'gamma': -8.0,     # Negative gamma (short options)
    'theta': 120.0,    # Positive theta (collecting premium)
    'vega': -450.0,    # Negative vega (short volatility)
    'rho': -25.0       # Small rate exposure
}

position_value = 50000  # Current position notional value

print('Portfolio Greeks:')
print('=' * 80)
for greek, value in portfolio_greeks.items():
    print(f"  {greek.capitalize():8s}: {value:+.2f}")
print(f"\nPosition Value: ${position_value:,.0f}")

In [ ]:
# Run stress test suite
print('\nStress Test Results:')
print('=' * 80)

stress_results = ScenarioTester.run_stress_test_suite(
    greeks=portfolio_greeks,
    position_value=position_value
)

# Display results
print(stress_results.to_string())

In [ ]:
# Identify worst scenarios
print('\nWorst Case Scenarios (Top 5):')
print('=' * 80)

worst_scenarios = ScenarioTester.identify_worst_scenarios(
    greeks=portfolio_greeks,
    position_value=position_value,
    top_n=5
)

for i, (name, result) in enumerate(worst_scenarios, 1):
    print(f"\n{i}. {name}")
    print(f"   P&L Estimate: ${result.pnl_estimate:+,.0f} ({result.pnl_percent:+.2%})")
    print(f"   Spot: {result.scenario.spot_change:+.0%}, Vol: {result.scenario.vol_change:+.0%}")

In [ ]:
# Apply specific historical scenarios
print('\nHistorical Scenario Analysis:')
print('=' * 80)

historical_events = ['black_monday_1987', 'lehman_2008', 'covid_crash_2020', 'volmageddon_2018']

for event in historical_events:
    scenario = HISTORICAL_SCENARIOS[event]
    result = ScenarioTester.apply_scenario(
        greeks=portfolio_greeks,
        position_value=position_value,
        scenario=scenario
    )
    print(f"\n{event.replace('_', ' ').title()}:")
    print(f"  Description: {scenario.description}")
    print(f"  Market Move: Spot {scenario.spot_change:+.0%}, Vol {scenario.vol_change:+.0%}")
    print(f"  P&L Impact: ${result.pnl_estimate:+,.0f} ({result.pnl_percent:+.2%})")

In [ ]:
# Sensitivity Analysis
print('\nSensitivity Analysis:')
print('=' * 80)

# Spot sensitivity
spot_sens = ScenarioTester.spot_sensitivity(
    greeks=portfolio_greeks,
    position_value=position_value,
    spot_range=(-0.10, 0.10),  # -10% to +10%
    num_points=21
)

print('\nSpot Price Sensitivity:')
print(spot_sens.to_string())

In [ ]:
# Volatility sensitivity
vol_sens = ScenarioTester.vol_sensitivity(
    greeks=portfolio_greeks,
    position_value=position_value,
    vol_range=(-0.30, 0.50),  # -30% to +50% vol change
    num_points=17
)

print('\nVolatility Sensitivity:')
print(vol_sens.to_string())

In [ ]:
# What-If Analysis: Spot vs Vol Matrix
print('\nWhat-If Analysis (Spot vs Vol):')
print('=' * 80)

whatif_matrix = ScenarioTester.what_if_spot_vol(
    greeks=portfolio_greeks,
    position_value=position_value,
    spot_range=(-0.10, 0.10),
    vol_range=(-0.20, 0.40),
    spot_steps=5,
    vol_steps=7
)

print('\nP&L Matrix ($ values):')
print(whatif_matrix.to_string())

In [ ]:
# Visualize What-If Matrix as Heatmap
fig = go.Figure(data=go.Heatmap(
    z=whatif_matrix.values,
    x=[f'{x:.0%}' for x in whatif_matrix.columns],
    y=[f'{y:.0%}' for y in whatif_matrix.index],
    colorscale='RdYlGn',
    zmid=0,
    text=[[f'${v:,.0f}' for v in row] for row in whatif_matrix.values],
    texttemplate='%{text}',
    textfont=dict(size=10),
    hoverongaps=False
))

fig.update_layout(
    title='P&L What-If Analysis: Spot Change vs Volatility Change',
    xaxis_title='Volatility Change',
    yaxis_title='Spot Price Change',
    height=500
)

fig.show()

In [ ]:
# Generate comprehensive stress report
print('\nComprehensive Stress Report:')
print('=' * 80)

report = ScenarioTester.generate_stress_report(
    greeks=portfolio_greeks,
    position_value=position_value
)

print(report)

## 9. Equity Curve and Visualizations

In [ ]:
# Create comprehensive performance dashboard
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        'Equity Curve',
        'Drawdown',
        'Daily Returns Distribution',
        'Cumulative Returns vs SPY'
    ),
    vertical_spacing=0.12
)

# Equity Curve
fig.add_trace(
    go.Scatter(
        x=pd.to_datetime(equity_curve['date']),
        y=equity_curve['equity'],
        mode='lines',
        name='Strategy Equity',
        line=dict(color='#2E86AB', width=2)
    ),
    row=1, col=1
)

# Add initial capital line
fig.add_hline(y=INITIAL_CAPITAL, line_dash='dash', line_color='gray', 
              annotation_text='Initial Capital', row=1, col=1)

# Drawdown
equity_series = pd.Series(equity_curve['equity'].values)
rolling_max = equity_series.expanding().max()
drawdown = (equity_series - rolling_max) / rolling_max * 100

fig.add_trace(
    go.Scatter(
        x=pd.to_datetime(equity_curve['date']),
        y=drawdown,
        fill='tozeroy',
        mode='lines',
        name='Drawdown %',
        line=dict(color='#A23B72', width=1)
    ),
    row=1, col=2
)

# Daily Returns Distribution
fig.add_trace(
    go.Histogram(
        x=daily_returns.values,
        nbinsx=50,
        name='Daily Returns',
        marker_color='steelblue'
    ),
    row=2, col=1
)

# Cumulative returns
cumulative_returns = (1 + daily_returns).cumprod() - 1
fig.add_trace(
    go.Scatter(
        x=cumulative_returns.index,
        y=cumulative_returns.values * 100,
        mode='lines',
        name='Cumulative Return %',
        line=dict(color='green', width=2)
    ),
    row=2, col=2
)

fig.update_layout(
    height=700,
    title_text=f'IV Rank Straddle Strategy Performance Dashboard',
    showlegend=True
)

fig.update_yaxes(title_text='Equity ($)', row=1, col=1)
fig.update_yaxes(title_text='Drawdown (%)', row=1, col=2)
fig.update_yaxes(title_text='Frequency', row=2, col=1)
fig.update_yaxes(title_text='Cumulative Return (%)', row=2, col=2)

fig.show()

## 10. Summary and Key Takeaways

In [ ]:
# Print comprehensive summary
print('=' * 80)
print('STRATEGY SUMMARY: IV Rank-Based Short Straddle')
print('=' * 80)

print('\n--- Strategy Rules ---')
print(f'  Entry: Sell ATM straddle when 90-day IV Rank > {IV_RANK_HIGH}% OR < {IV_RANK_LOW}%')
print(f'  DTE: {MIN_DTE}-{MAX_DTE} days')
print(f'  Exit: 50% profit target, 200% stop loss, or 7 DTE')

print('\n--- Backtest Results ---')
print(f'  Period: {START_DATE.date()} to {END_DATE.date()}')
print(f'  Initial Capital: ${INITIAL_CAPITAL:,.0f}')
print(f'  Final Equity: ${results["final_equity"]:,.2f}')
print(f'  Total Return: {results["total_return"]:.2%}')
print(f'  Annualized Return: {perf.get("annualized_return", 0):.2%}')

print('\n--- Risk Metrics ---')
print(f'  Sharpe Ratio: {perf.get("sharpe_ratio", 0):.3f}')
print(f'  Sortino Ratio: {perf.get("sortino_ratio", 0):.3f}')
print(f'  Max Drawdown: {perf.get("max_drawdown", 0):.2%}')
print(f'  Win Rate: {perf.get("win_rate", 0):.1f}%')

print('\n--- Monte Carlo Insights ---')
ci_return = confidence_intervals.get('total_return')
if ci_return:
    print(f'  95% CI for 1-Year Return: [{ci_return.lower_bound:.2%}, {ci_return.upper_bound:.2%}]')
print(f'  Monte Carlo VaR (95%): {var_metrics["95%"]["var"]:.2%}')
print(f'  Monte Carlo CVaR (95%): {var_metrics["95%"]["cvar"]:.2%}')

print('\n--- Stress Test Insights ---')
print(f'  Worst Historical Scenario: {worst_scenarios[0][0]}')
print(f'  Worst Case P&L: ${worst_scenarios[0][1].pnl_estimate:+,.0f} ({worst_scenarios[0][1].pnl_percent:+.2%})')

print('\n' + '=' * 80)
print('Analysis complete!')

## 11. Clean Up

In [ ]:
# Close database connection
adapter.disconnect()
print('Database connection closed.')